In [53]:
import numpy as np
import pandas as pd
import cvxpy as cp
from geopy import distance

In [2]:
data = pd.read_csv('../Dataset/fri26_d.txt',delim_whitespace=True,header=None).to_numpy()

In [65]:
points = [(-12.059296, -76.975893),
          (-12.079575, -77.009686),
          (-12.087303, -76.996620),
          (-12.084391, -76.975651),
          (-12.063603, -76.960483),
          (-12.056762, -77.014452),
          (-12.011531, -77.002383)]

################################################
# Building distance matrix
################################################
n = len(points)
C = np.zeros((n,n))

for i in range(0, n):
    for j in range(0, len(points)):
        C[i,j] = distance.distance(points[i], points[j]).km

# Showing distance matrix
print('Distance Matrix is:\n')
print(np.round(C,4))

Distance Matrix is:

[[0.     4.3092 3.8329 2.7762 1.7441 4.2076 6.0199]
 [4.3092 0.     1.6596 3.7435 5.6408 2.5764 7.5691]
 [3.8329 1.6596 0.     2.3055 4.7278 3.8966 8.4056]
 [2.7762 3.7435 2.3055 0.     2.8311 5.2142 8.5694]
 [1.7441 5.6408 4.7278 2.8311 0.     5.9246 7.3483]
 [4.2076 2.5764 3.8966 5.2142 5.9246 0.     5.1733]
 [6.0199 7.5691 8.4056 8.5694 7.3483 5.1733 0.    ]]


In [66]:
# Generate a random SDP.
n = 7
np.random.seed(1)
# C = data
e = np.ones((n,1))
I = np.eye(n)
J = np.ones((n,n))
constant =  2-2*np.cos((2*np.pi)/n)
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [
    X[i][i] == 0 for i in range(n)
]
constraints += [
    X[i][j] >= 0 for i in range(n) for j in range(n)
]
constraints += [
    X[i][j] <= 1 for i in range(n) for j in range(n)
]
constraints += [
    2*I-X+J-(constant*I) >> 0
]
prob = cp.Problem(cp.Minimize(1/2*cp.trace(C @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 1.6615081465717716e-08
A solution X is
[[ 7.41866007e-18  2.39734272e-08  4.15035659e-08  9.70291983e-08
   1.15217402e-07 -2.63303463e-08  5.64465212e-08]
 [ 2.39734272e-08 -5.60523879e-17  1.33589270e-07  5.75713564e-08
   4.35799864e-08  4.14197542e-08 -7.35571351e-08]
 [ 4.15035659e-08  1.33589270e-07 -1.43194356e-17  9.26875423e-08
   8.13855274e-08  2.59943217e-08 -2.07956635e-07]
 [ 9.70291983e-08  5.75713564e-08  9.26875423e-08 -4.94985732e-18
   1.05091404e-07 -2.02406115e-08 -2.25056235e-07]
 [ 1.15217402e-07  4.35799864e-08  8.13855274e-08  1.05091404e-07
  -1.86309138e-17 -1.70984353e-08  1.38905935e-08]
 [-2.63303463e-08  4.14197542e-08  2.59943217e-08 -2.02406115e-08
  -1.70984353e-08  1.60086328e-18  3.10433601e-07]
 [ 5.64465212e-08 -7.35571351e-08 -2.07956635e-07 -2.25056235e-07
   1.38905935e-08  3.10433601e-07 -6.04877215e-18]]
